In [8]:
import json

import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from torchvision.models import ResNet
from custom_image_folder import CustomImageFolder
import chromadb

# Pretrained ResNet18
In this script a  pretrained ResNet18 is used to obtain embbedings for all Cifar10 samples.
With the embeddings of the training samples the test samples are classified via KNN.
The accuracy of that KNN classifier on the test set is recorded.
## Prepare embedding model

In [9]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18')
#torch.save(model.state_dict, "/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/embeddings/pretrained_models/resnet18_pretrained")

Using cache found in /home/erik/.cache/torch/hub/pytorch_vision_v0.10.0


In [10]:
print(F"type of model {type(model)}")
print(f"{model.fc}")

type of model <class 'torchvision.models.resnet.ResNet'>
Linear(in_features=512, out_features=1000, bias=True)


In [11]:
# Replace last FC layer by identity mapping so that the model outputs vectors of dim 512
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

model.fc = Identity()

## Prepare vector store

In [12]:
chroma_client = chromadb.PersistentClient(path="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/embeddings/embedding_data/embedding_store")

collection = chroma_client.create_collection(name="pretrained_resnet18_embeddings")
#collection = chroma_client.get_collection(name="my_collection")
print(f"Samples in db: {len(collection.get()['embeddings'])}")

TypeError: object of type 'NoneType' has no len()

In [13]:
def get_embeddings_and_store(embedding_model, data_loader, chroma_collection, dataset_type):
    count = 0
    for batch in data_loader:
        print(count)
        batch_paths_and_images, batch_labels = batch
        batch_labels = batch_labels.tolist()
        batch_paths = list(batch_paths_and_images[0])
        batch_images = batch_paths_and_images[1]
        
        out = embedding_model(batch_images)
        
        chroma_collection.add(
            embeddings = out.tolist(),
            metadatas = [{'path':x[0], 'label':x[1], 'dataset_type':dataset_type} for x in zip(batch_paths, batch_labels)],
            ids= [dataset_type + str(x) for x in list(range( count * 256, count * 256 + len(batch_labels)))]
        )

        count += 1


## Get embeddings of train dataqset

In [14]:
# Use custom Dataset that returns image path in get item
train_dataset = CustomImageFolder(
    root="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/train",
    transform=ToTensor()
)
# Create DataLoader
train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)
# feed through embedding model
model.eval()
get_embeddings_and_store(model, train_data_loader, collection, dataset_type="train")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [15]:
print(f"Samples in db: {len(collection.get()['ids'])}")
print(collection.get()['ids'][0])
print(collection.get()['ids'][-1])

Samples in db: 50000
train0
train49999


## Get embeddings of test dataset

In [16]:
# Use custom Dataset that returns image path in get item
test_dataset = CustomImageFolder(
    root="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/test",
    transform=ToTensor()
)
# Create DataLoader
test_data_loader = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)
# feed through embedding model
get_embeddings_and_store(model, test_data_loader, chroma_collection=collection, dataset_type="test")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [17]:
print(f"Samples in db: {len(collection.get()['ids'])}")
print(collection.get()['ids'][50000])
print(collection.get()['ids'][-1])

Samples in db: 60000
test0
test9999


In [18]:
collection.get(include=['embeddings', 'documents', 'metadatas'])['embeddings'][0]

[0.009699023328721523,
 0.17504151165485382,
 0.007887656800448895,
 0.0,
 0.5267356038093567,
 0.0016377257416024804,
 0.009823333472013474,
 0.0,
 0.20158956944942474,
 0.6484022736549377,
 0.8433561325073242,
 0.018825815990567207,
 0.39665666222572327,
 0.05148020386695862,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006327408831566572,
 0.13787715137004852,
 0.32102450728416443,
 0.03618595749139786,
 0.4599781632423401,
 0.21071012318134308,
 0.0,
 0.0,
 0.0,
 0.5283924341201782,
 0.027640417218208313,
 0.5694782137870789,
 0.15610173344612122,
 0.4104174077510834,
 0.0002860734239220619,
 0.13793927431106567,
 0.2193525731563568,
 0.3740973174571991,
 0.03476358577609062,
 0.5137283802032471,
 0.0,
 0.02763727679848671,
 0.17142429947853088,
 0.028841199353337288,
 0.2910490930080414,
 0.0007112201419658959,
 0.048084963113069534,
 0.19917146861553192,
 0.4004630148410797,
 0.18925559520721436,
 0.11410537362098694,
 0.18867990374565125,
 0.6121984720230103,
 0.0,
 0.0,
 0.0120031852275133